In [98]:
from data_loader import *
from model import *
from loss import *

from torch.utils.data import Dataset, DataLoader
from PIL import Image
import cv2
from torchvision import transforms
import os
from torch.autograd import Function
from torch import nn
import matplotlib.pyplot as plt

In [99]:
root_path = '/home/renyan/ossdata/massachusetts-roads-dataset/'
road_path = root_path + "tiff_select2_parts_16/"

classes = ['background', 'road']
colormap = [[0 , 0, 0], [255, 255, 255]]

INPUT_SIZE, OUTPUT_SIZE = 256, 256

In [ ]:
train_dataset = data_loader.RoadDataset(road_path, True, INPUT_SIZE, OUTPUT_SIZE)
val_dataset = data_loader.RoadDataset(road_path, False, INPUT_SIZE, OUTPUT_SIZE)